In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self,x,y):
        self.x = x
        self.y = y
        out= x * y
        return out 
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    

In [3]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out 
    def backward(self, dout):
        dx = dout * 1 
        dy = dout * 1 
        return dx, dy
    

In [4]:
apple = 100
apple_num = 2
orange = 150 
orange_num = 3
tax = 1.1
#layer 
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()


In [5]:
#forward
apple_price = mul_apple_layer.forward(apple,apple_num) 
orange_price = mul_orange_layer.forward(orange, orange_num) 
all_price = add_apple_orange_layer.forward(apple_price,orange_price)
price = mul_tax_layer.forward(all_price, tax)

In [9]:
#backward 
dprice = 1 
dall_price, dtax = mul_tax_layer.backward(dprice)  
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num =  mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [10]:
print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:
### параметрүүд
# Хэдэн удаа оролтын датагаар моделийг сургах
EPOCH = 1            
# 1 удаад оруулах оролтын датаны хэмжээ
BATCH_SIZE = 50
# Learning Rate
LR = 0.001              
# MNIST датаг татах эсэх 
DOWNLOAD_MNIST = True  

In [67]:
# Mnist digits dataset
batch_size = 32
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,                                     
    # this is training data
    transform=torchvision.transforms.ToTensor(),    
    # torch.FloatTensor of shape (Color x Height x Width) and 
    # normalize in the range [0.0, 1.0]
    download=DOWNLOAD_MNIST,                        
)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_data = torchvision.datasets.MNIST(root='./mnist', train=False,
                                       download=True, transform=torchvision.transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [68]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [69]:
# plot example
fig = plt.figure(figsize=(9, 15))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05,
                    wspace=0.05)

for i in range(9):
    ax = fig.add_subplot(1, 9, i + 1, xticks=[], yticks=[])
    ax.imshow(train_data.data[i].numpy(), cmap='gray')

In [70]:
# plot one example
print(train_data.data.size())                 # (60000, 28, 28)
print(train_data.targets.size())               # (60000)
     
torch.Size([60000, 28, 28])
torch.Size([60000])

torch.Size([60000, 28, 28])
torch.Size([60000])


torch.Size([60000])

In [71]:
test_x = Variable(torch.unsqueeze(test_data.data, dim=1)).type(torch.FloatTensor)[:2000]/255.   
# shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.targets[:2000]

In [72]:
# implementation
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, padding=2),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=None, padding=0),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=2),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=32*7*7,out_features=50, bias=False),
            nn.ReLU(),
            nn.Linear(in_features=50, out_features=10, bias=True)
        ) 
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0),-1)
        output = self.fc(x)
        return output, x
     

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
device

device(type='cuda')

In [74]:
cnn = CNN().cuda()
print(cnn)  

CNN(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1568, out_features=50, bias=False)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)


In [75]:
from torchinfo import summary
stats = summary(cnn,(1,1,28,28),verbose=2)

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [1, 10]                   --
├─Sequential: 1-1                        [1, 32, 7, 7]             --
│    └─0.weight                                                    ├─400
│    └─0.bias                                                      ├─16
│    └─3.weight                                                    ├─12,800
│    └─3.bias                                                      └─32
│    └─Conv2d: 2-1                       [1, 16, 28, 28]           416
│    │    └─weight                                                 ├─400
│    │    └─bias                                                   └─16
│    └─ReLU: 2-2                         [1, 16, 28, 28]           --
│    └─MaxPool2d: 2-3                    [1, 16, 14, 14]           --
│    └─Conv2d: 2-4                       [1, 32, 14, 14]           12,832
│    │    └─weight                                            

In [76]:
optimizer = torch.optim.SGD(cnn.parameters(), lr=LR                                                                                                                                                     )   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss() 

In [77]:
# for visualization
from matplotlib import cm
from sklearn.manifold import TSNE

def plot_with_labels(lowDWeights, labels):
    plt.cla()
    X, Y = lowDWeights[:, 0], lowDWeights[:, 1]
    for x, y, s in zip(X, Y, labels):
        c = cm.rainbow(int(255 * s / 9)); plt.text(x, y, s, backgroundcolor=c, fontsize=9)
    plt.xlim(X.min(), X.max()); plt.ylim(Y.min(), Y.max()); plt.title('Visualize last layer');
    plt.show(); 
    #plt.pause(0.01)

In [78]:
plt.ion()

In [79]:
# training
test_x = test_x.to(device)
test_y = test_y.to(device)
for epoch in range(EPOCH):
    print(epoch)
    print("----------------------------------------------------------------------")
    for step, (x, y) in enumerate(train_loader):
        optimizer.zero_grad
        # clear gradients for this training step
        x = x.to(device)
        y = y.to(device)

        # forward pass
        output =  cnn(x)[0]
        loss = loss_func(output,y)    
        # backward pass
        loss.backward()
        optimizer.step()
        #Visualization
        if step % 300 == 0:
            test_output, last_layer = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data
            accuracy = (pred_y == test_y).sum().item() / float(test_y.size(0))
            print('step:{}'.format(step), '| loss:{:.4f}'.format(loss.data), '| test accuracy: {:.2f}'.format(accuracy) ) 

            # # Visualization of trained flatten layer (T-SNE)
            # tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
            # plot_only = 500
            # low_dim_embs = tsne.fit_transform(last_layer.cpu().data.numpy()[:plot_only, :])
            # labels = test_y.cpu().numpy()[:plot_only]
            # plot_with_labels(low_dim_embs, labels)
plt.ioff()

0
----------------------------------------------------------------------
step:0 | loss:2.2925 | test accuracy: 0.13
step:300 | loss:0.7368 | test accuracy: 0.81
step:600 | loss:0.6877 | test accuracy: 0.89
step:900 | loss:0.4953 | test accuracy: 0.78
step:1200 | loss:0.9766 | test accuracy: 0.74
step:1500 | loss:1.4643 | test accuracy: 0.66
step:1800 | loss:1.4993 | test accuracy: 0.50


In [80]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for x, y in test_loader:
        # calculate outputs by running images through the network 
        x = x.to(device)
        y = y.to(device)
        outputs = cnn(x)[0]
        # the class with the highest energy is what we choose as prediction
        predicted = torch.max(outputs.data, 1)[1]
        total += y.size(0)
        correct += (predicted == y).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 55 %


In [81]:
# print 10 predictions from test data
test_output, _ = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].cpu().data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10].cpu().numpy(), 'real number')

[7 2 8 5 4 1 4 5 9 9] prediction number
[7 2 1 0 4 1 4 9 5 9] real number


In [82]:

cnn.conv[0].weight.grad[0]
cnn.conv[0].weight[0]
cnn.conv[0].bias.grad
cnn.conv[0].bias

Parameter containing:
tensor([-16.2925,   0.6440, -11.4559,  -6.4433,  -8.6575,  -2.5533,  -8.0696,
        -10.3240,  -7.3460, -12.1773,  -6.2775,  -5.2502,  -4.9110,  -5.8757,
         -3.4010, -12.0178], device='cuda:0', requires_grad=True)